<a href="https://colab.research.google.com/github/devangi2000/Deep-Learning/blob/master/Learning_Rate_scheduler_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn # All neural net modules, nn.Linear, nn.Conv2d, BatchNorm, Loss Functions
import torch.optim as optim # For all optimization algos, Adam, SGD, etc.
import torch.nn.functional as F # All functions that don't have any parameters
from torch.utils.data import DataLoader # Gives easier dataset management and creates mini batches
import torchvision.datasets as datasets # Standard datasets we can import in an easy way
import torchvision.transforms as transforms # Transformations we can perform on our datasets
import torchvision

# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
num_classes = 10
learning_rate = 0.1
batch_size = 1024
num_epochs = 100

model = torchvision.models.googlenet(pretrained=True)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


In [3]:
for param in model.parameters():
  param.requires_grad = False

model.fc = nn.Linear(1024, num_classes)
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [4]:
# Load Data
train_dataset = datasets.CIFAR10(root='dataset/', train=True,
                                 transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=5, verbose=True)

Extracting dataset/cifar-10-python.tar.gz to dataset/


In [5]:
# TRAIN NETWORK
for epoch in range(1, num_epochs):
  losses = []

  for batch_idx, (data, targets) in enumerate(train_loader):
    # Get data to cuda if possible
    data = data.to(device=device)
    targets = targets.to(device=device)

    # forward
    scores = model(data)
    loss = criterion(scores, targets)

    losses.append(loss.item())

    #backward
    loss.backward()

    #gradient descent or adam step
    optimizer.step()
    optimizer.zero_grad()

  mean_loss = sum(losses)/len(losses)
  scheduler.step(mean_loss)
  print(f'Cost at epoch {epoch} is {mean_loss}')


Cost at epoch 1 is 12.068784704013746
Cost at epoch 2 is 2.765654354679341
Cost at epoch 3 is 2.881890131502735
Cost at epoch 4 is 2.4486031678258158
Cost at epoch 5 is 3.2674944936012733
Cost at epoch 6 is 2.949040699978264
Cost at epoch 7 is 2.984113766222584
Cost at epoch 8 is 3.0730144053089377
Cost at epoch 9 is 3.2093954475558535
Epoch    10: reducing learning rate of group 0 to 1.0000e-02.
Cost at epoch 10 is 3.4419242313929965
Cost at epoch 11 is 2.4000180838059406
Cost at epoch 12 is 1.9949233823893022
Cost at epoch 13 is 1.9006336684129677
Cost at epoch 14 is 1.841115547686207
Cost at epoch 15 is 1.8096624028925994
Cost at epoch 16 is 1.7882455514401805
Cost at epoch 17 is 1.7789379552919038
Cost at epoch 18 is 1.7657601492745536
Cost at epoch 19 is 1.7647692086745281
Cost at epoch 20 is 1.7623900515692574
Cost at epoch 21 is 1.7591224981814015
Cost at epoch 22 is 1.7569553949395005
Cost at epoch 23 is 1.7609127662619766
Cost at epoch 24 is 1.767488949152888
Cost at epoch 25